# Assignment (5) - ForestFires

In [1]:
# Importing necassery libraries for data manipulation.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importing necassery Libraries for making the model and evaluating it.
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# Reading the csv file of the dataset into a data frame using 'read_csv' from pandas library.
df = pd.read_csv(r"C:\Users\khale\Documents\Sprints\Attachment NN\forestfires.csv")

# Showing the shape of the dataset.
print(f"DataSet Shape: ", df.shape)

# Showing the first five rows of the dataset.
df.head()

DataSet Shape:  (517, 13)


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [3]:
# Showing the main information about the dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   month   517 non-null    object 
 3   day     517 non-null    object 
 4   FFMC    517 non-null    float64
 5   DMC     517 non-null    float64
 6   DC      517 non-null    float64
 7   ISI     517 non-null    float64
 8   temp    517 non-null    float64
 9   RH      517 non-null    int64  
 10  wind    517 non-null    float64
 11  rain    517 non-null    float64
 12  area    517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


- We can see that the data types of the columns are good.
- And that our dataset doesn't seem to have null values But we will need to be sure.

In [4]:
# Caluclating the number of null values in each column.
df.isnull().sum()

X        0
Y        0
month    0
day      0
FFMC     0
DMC      0
DC       0
ISI      0
temp     0
RH       0
wind     0
rain     0
area     0
dtype: int64

- We are 100% Sure now that our data is clean of null values.

In [5]:
# Calculating the number of duplicated rows in the data.
print(f"Number Of duplicated rows in the dataset: {df.duplicated().sum()}")

Number Of duplicated rows in the dataset: 4


In [6]:
# Dropping the duplicated rows from the dataset with "drop_duplicates" function.
df.drop_duplicates(inplace = True)
# Calculating the number of duplicated rows in the data after dropping.
print(f"Number Of duplicated rows in the dataset after dropping: {df.duplicated().sum()}")

Number Of duplicated rows in the dataset after dropping: 0


In [7]:
# Showing the main charactaristics of the columns in our dataset.
df.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
count,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000
mean,4.678363,4.300195,90.637427,110.989279,549.155166,9.023587,18.887524,44.348928,4.010526,0.021832,12.891598
std,2.320059,1.230925,5.540920,64.191013,247.724594,4.575543,5.828128,16.358803,1.796613,0.297107,63.892586
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,3.000000,4.000000,90.200000,68.600000,440.900000,6.400000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,4.000000,4.000000,91.600000,108.300000,664.500000,8.400000,19.300000,42.000000,4.000000,0.000000,0.540000
75%,7.000000,5.000000,92.900000,142.400000,713.900000,11.000000,22.800000,53.000000,4.900000,0.000000,6.570000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


- So Actually Our data scales are good and saisfies the ranges given (In the description) for this columns.
- But there is some 0 values in the  'area' column that we should handle.

In [8]:
# Applying log transformation on the data column with many 0 values.
df['area_log'] = np.log10(df['area'] + 1)

- Applying log transformation to the 'area' column to make the range of values better and adding 1 to prevent the erroes and infinite values generated from taking the log to zero.

## Data Analysis

- What is the month that have the highest average tempreture that can lead to a fire?

In [9]:
# Grouping each month with its average tempreture and sorting it.
month_temp = df.groupby("month")["temp"].mean().sort_values(ascending = False)
month_temp.head()

month
jul    22.109375
aug    21.648352
jun    20.575000
sep    19.612209
oct    17.093333
Name: temp, dtype: float64

- We can see that july, august and june has the top 3 average tempreture So we should enhance our care for this times.

- What is the average area fired for each day of the weak? 

In [10]:
# Grouping each day with its average area and sorting it.
day_wind = df.groupby("day")['area'].mean().sort_values(ascending = False)
day_wind

day
sat    25.496386
thu    16.345902
tue    12.621719
wed    10.916981
sun    10.212021
mon     9.547703
fri     5.324286
Name: area, dtype: float64

- We want to see the average measures for each unqiue coordinate's combination.

In [11]:
# Grouping Each combination of coordinates with the average measures of the rest of the columns in the data.
coordinates_df = df.groupby(['X', 'Y'])[['wind', 'temp', 'DMC', 'FFMC', 'DC', 'ISI', 'rain', 'area']].mean().sort_values("area", ascending = False)

# Printing the first 5 rows in this grouped oordinates data frame.
coordinates_df.head()

,,wind,temp,DMC,FFMC,DC,ISI,rain,area
X,Y,,,,,,,,
8,8,4.50,26.200,191.400,91.70,635.900,7.80,0.0,185.7600
9,4,2.90,24.225,82.700,89.70,460.375,7.85,0.0,46.4025
7,6,6.50,19.750,181.750,92.15,434.250,11.75,0.1,43.2250
9,6,2.70,20.500,248.400,91.60,753.800,6.30,0.0,42.8700
1,5,2.55,25.000,137.675,92.20,707.275,7.55,0.0,28.8675


## ML Model - Neural Network

### Feature Processing & Selection

- As our data has two categorical columns (day, month) we need to encode it. 
- We will use get_dummies function to encode this categorical columns.

In [12]:
# Encoding the categoris to numerical values using 'get_dummies' from pandas 
# returning a new data frame with the numerical columns for each categorical feature.
df = pd.get_dummies(df, columns = ['day', 'month'], drop_first = True)

# Displaying the first five rows from the data after encoding.
df.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,0,1,0,0,0,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,0,1,0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,0,1,0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,0,1,0,0,0,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,0,1,0,0,0,0


### Shuffling the data

In [13]:
# generating random indices from the length of the data to shuffle with it.
shuffle_order = np.random.permutation(len(df))

# Shuffling the dataset to overcome the orderd values in the data.
shuffled_df = df.iloc[shuffle_order, :]

In [14]:
# Extracting the feature matrix and target vector from the shuffled data.
X = shuffled_df.drop(columns = ['area', 'area_log'])
y = shuffled_df['area_log']

# Split the data into training and testing data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaling the data to be all at the same scale.
scaler = StandardScaler()
# Transforming and scaling training features
X_train_scaled = scaler.fit_transform(X_train)
# Transforming and scaling test features
X_test_scaled = scaler.transform(X_test)

### Constructing the model

- The NN model will contain 3 layers:
-- First layer will have 32 neurons and activation function 'relu' and specifying the input shape of the vector of features.
-- Second layer will have 16 units and activation function 'relu'
-- Fourth layer will have 32 units and no activation function (linear).
----------------------------------------
- Will compile the model with optimization function 'Gradient Descent' with learning rate 0.01 and loss function 'mean squared error'.
- Then will fit the model to the scaled training matrix and target vector and setting the number of epochs to 50.

In [20]:
# Initializing the model and constructing its layers.
nn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units = 32, activation = "relu", input_shape = (X_train_scaled.shape[1], )),
    tf.keras.layers.Dense(units = 16, activation = "relu"),
    tf.keras.layers.Dense(units = 1)
])

# Compling the model speifing its optimizer for updating the weights and setting its learning rate. 
# and its loss function to compute the error.
nn_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01), loss = "mean_squared_error")

# Training the model on the scaled training data and determining teh number of epochs to be 50.
nn_model.fit(X_train_scaled, y_train, epochs = 50)

Epoch 1/50
13/13 [==============================] - 0s 1ms/step - loss: 0.9656
Epoch 2/50
13/13 [==============================] - 0s 1ms/step - loss: 0.5375
Epoch 3/50
13/13 [==============================] - 0s 1ms/step - loss: 0.4365
Epoch 4/50
13/13 [==============================] - 0s 1ms/step - loss: 0.4028
Epoch 5/50
13/13 [==============================] - 0s 2ms/step - loss: 0.3889
Epoch 6/50
13/13 [==============================] - 0s 1ms/step - loss: 0.3819
Epoch 7/50
13/13 [==============================] - 0s 2ms/step - loss: 0.3765
Epoch 8/50
13/13 [==============================] - 0s 1ms/step - loss: 0.3725
Epoch 9/50
13/13 [==============================] - 0s 2ms/step - loss: 0.3689
Epoch 10/50
13/13 [==============================] - 0s 2ms/step - loss: 0.3662
Epoch 11/50
13/13 [==============================] - 0s 2ms/step - loss: 0.3636
Epoch 12/50
13/13 [==============================] - 0s 2ms/step - loss: 0.3609
Epoch 13/50
13/13 [==============================

### Evaluate the model

In [23]:
# Testing the model by Predicting on the test data.
y_pred = nn_model.predict(X_test_scaled)

# Calculating MSE and MAE between the test data and the predicted values.
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Printing MSE and MAE.
print(f'Mean Squared Error (MSE): {mse}')

4/4 [==============================] - 0s 1ms/step
Mean Squared Error (MSE): 0.3631332242403747
